# Deep Learning Part 3: Datasets, Data Loading, Cross Entropy, and Convolutional Networks

## Grading Standards:
*  10%: Dataset/MNIST section
*  10%: Correct implementation and use of cross entropy loss
*  20%: Correct training/validation functions
*  40%: Successful training and validation with MLP and convolution networks
*  12%: Convolutional layer quiz
*   8%: Comparison between MLP and convolution networks
___


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

Set your global variable `device`, using the `torch.device()` function. In order to use cuda remember to request a GPU from Runtime > Change Runtime.

***Important Note**: If you spend too much time or memory on the GPU in Google Colab then you will be timed out. This may not be a big deal with this lab, but it can become a big deal in later labs. It is recommended to set your `device` and Runtime to CPU first and once everything in the lab is working properly to set it to the GPU.*

In [ ]:
device = None

---

# Datasets and Data Loading



In [ ]:
from torch.utils.data import Dataset, DataLoader

We are going to make a PyTorch `Dataset`.
There are three parts to creating a `Dataset`:
1. `__init__()`: This is where you get all relevant data for your dataset.
2. `__len__()`: You return how large your dataset is.
3. `__getitem__()`: You return an item from your dataset given an index.

Implement the TODOs below.

In [ ]:
class SineDataset(Dataset):
    def __init__(self):
        # TODO: The code is same from lab 2, so you can uncomment the code below.
        # self.x = torch.rand((100,1))*8 - 4
        # self.y = torch.sin(self.x) + torch.randn_like(self.x)*.1 # the second part of the sum adds noise to the function

    def __len__(self):
        # TODO: Return the len of your dataset.
        pass

    def __getitem__(self, i):
        # TODO: i will be an index so return x_i and y_i
        pass

We will now create a `SineDataset` and print out the length of dataset, i.e. `len(dataset)`, and the item in your dataset at index 0, i.e. `dataset[0]`.

*Note: `__len__()` and `__getitem__()` are private methods and should not be called directly.*

A `DataLoader` uses the `__len__` and `__getitem__` of a `Dataset` to sample indices in `[0, ..., len(dataset)-1]` and collect a batch of items from the `Dataset`.
The `DataLoader` will then try to convert the sampled entries into tensors (if they are not already) and concatenate them together.
Create a `DataLoader` object below; pass in your dataset and `batch_size=32` as arguments.

Iterate through your dataloader with a for loop. Because `SineDataset.__getitem__()` returns two items, the for loop will return a tuple.
Either unpack the entries in your tuple in the for loop:
```python
for x, y in dataloader
```
or after the loop:
```python
for batch in dataloader:
    x, y = batch
```

Print out the shapes of `x` and `y` for each batch in the dataloader.

You will notice that the shapes are `(B, Z_in)`, where `B` is batch size and `Z_in` is our input feature size, which is exactly what we want.
Also note that the last batch has a batch size of 4; this is because the `DataLoader` samples **without replacement** and these are the last items in our `Dataset` that have not been sampled.

Let's now create our real dataset.

---

# MNIST

We are now going to look at the MNIST dataset, which is a dataset of handwritten numbers.
Our objective will be to create a neural network that can predict the number given the image.

First import `torchvision` below so we can retrieve the MNIST dataset.

In [ ]:
import torchvision

You can use `torchvision.datasets.MNIST()` to download the MNIST dataset (which inherits the `Dataset` class).
For arguments, specify `root="/tmp/"` to denote the location, `train=True` or `train=False` to get the training or test dataset, `download=True` to specify you want to download the dataset, and `transform=torchvision.transforms.ToTensor()` to convert the MNIST images from PIL images to PyTorch tensors.
Create both a `train_dataset` and `val_dataset`.

*Note: It is good practice to use a train, val, and test dataset, especially in the real world, but in this class we will mainly focus on train and val datasets to simplify things.*

Print out the lengths of `train_dataset` and `val_dataset`.

Grab element 0 from the `train_dataset`. As a heads up, like our `SineDataset`, `MNIST` returns an image `x` and a class/target `y`.

Use the `type()` function to see what type of object `x` and `y` are.

Since x is a tensor, print out its `.dtype`, `.shape`

The shape of a tensor image is `(C, H, W)`, where `C` is channels, `H` is height, and `W` is width.
In our case `x` has 1 channel and it is a 28x28 image.
Because there is 1 channel, it is likely that the image is grayscale (which it is).

Now visualize the image and display its class.
Use the `plt.imshow()` function to visualize `x`; add the argument `cmap="gray"` to denote the image is grayscale.
Use the `plt.title()` function to set the title of the `plt` image to the class `y`.

*Note: `plt` expects grayscale images to only have to dimension, HxW, so `.squeeze()` the 0th dimension*

Print out the min and max values of `x` using `torch.min()` and `torch.max()`.

Our tensors are normalized between 0 and 1, which is good so we don't have to do any normalization.
Now that we have a better understanding of our image data, let's examine the classes in the dataset.

Create a `get_dataset_classes()` function which takes a dataset as input and count how many times each class appears.
Return a dictionary where the keys are the classes and the values represent the number of times each class appears in the dataset.

In [ ]:
def get_dataset_classes(dataset):
    pass

Execute the code below to visualize the dataset classes.

In [ ]:
train_classes = get_dataset_classes(train_dataset)
val_classes = get_dataset_classes(val_dataset)

def plot_classes(ax, classes, title):
    ax.bar(classes.keys(), classes.values())
    ax.set_title(title)
    ax.set_xticks(list(range(10)))

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
plot_classes(axes[0], train_classes, "Train Dataset Classes")
plot_classes(axes[1], val_classes, "Val Dataset Classes")
plt.show()

There are 10 classes, which makes sense because there are 10 digits, 0-9.
Furthermore, the classes are pretty evenly spread on both datasets, which means our baseline accuracy is roughly 10%.

Now create two `DataLoader` objects called, `train_loader` and `val_loader`.
The `train_loader` should have your `train_dataset` and the `val_loader` should have your `val_dataset`.
Set the `batch_size` of both dataloaders equal to 32 and set `shuffle=True` for the `train_loader` so that the dataset is shuffled every time.
To improve the speed at which you `DataLoader`s can load the data, set `num_workers=4` (for multiprocessing).

In [ ]:
train_loader = None
val_loader = None

Now use `x, y = next(iter(train_loader))` to get a single batch of data from the `train_loader` and print out the shapes and dtypes of `x` and `y`

The shape of `x` is `(B, C, H, W)`, where `B` is the batch size. **Always remember, in PyTorch, your data should have a batch dimension.**
The shape of `y` is `(B,)` and it is a tensor of type `long`, which is what we want because we are doing classification, which means we want our target to be a class label.

___

### MLP Network

We are now going to make our network.
Because we are doing image classification, the input to our network is a batch of images, `shape=(B, C, H, W)`, and the output of our network is a batch of probabilities, `shape=(B, K)`, where `K` represents the number classes in our dataset.
In our case `K=10`.

We will first try to solve this problem using a fully connected deep network (like your DeepNet from lab 2), sometimes called a Multi-Layer Perceptron (MLP).  
Implement an `MLP` below (don't forget to use `nn.Sequential`, `nn.Linear`, `nn.ReLU`).
Because `nn.Linear` expects tensors of shape `(B, Z)`, where `Z` is the input feature size, we need to flatten our images. Use the `.view()` function to reshape `x.shape=(B, C, H, W)` into `x.shape=(B, Z)`, where `Z=C*H*W`.  

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_features, out_features, hidden_size):
        pass

    def forward(self, x):
        pass

Because we are doing classification, we need to use a different loss function that MSE; cross entropy loss is a good choice and is a common loss function for classification.

---

# Softmax and CrossEntropy

The cross entropy (CE) function is $CE(p, q) = - \sum p(x) \log q(x)$, where $p$ and $q$ are probability functions ($p$ is the target probabilities and $q$ is the predicted probabilities) and in our case $x$ represents a class.

$p$ represents the target distribution, the true class distribution, which means it is a one-hot vector $p_c$, where $c$ represents the index of the class:
$$p_i = \begin{cases}
1, & \textrm{if } i = c \\
0, & \textrm{if } i \not = c \\
\end{cases}$$

Then $CE(p, q) = - \sum_i p_i \log q_i$ will become $CE(p, q) = - p_c \log q_c$, because $p_{i \not = c} = 0$, which is further reduced to $CE(p, q) = - \log q_c$, since $p_c = 1$. In other words, cross entropy loss for classification is the negative log of the predicted probability of the correct class.

Therefore, $p$ is never passed into `F.cross_entropy_loss()`, instead you pass in $q$ you predicted distribution and $c$ the index of the correct class.

Implement `prenormalized_cross_entropy_loss` below. $q$ is assumed to be a normalized probability distribution.

*Note 1: Do **not** use a for loop. You can index into a tensor with array slicing (hint: You will need to use `torch.arange()` for the 0th dimension of `q`)*

*Note 2: Compute the mean cross entropy of the batch not the sum* 

In [ ]:
def prenormalized_cross_entropy_loss(q, c):
    pass

Validate your function below.

In [ ]:
def test_prenormalized_cross_entropy_loss():
    q = torch.tensor([[.1, .5, .4],
                      [.2, .2, .6],
                      [.3, .3, .3]])
    c = torch.tensor([2, 0, 1])
    assert torch.allclose(prenormalized_cross_entropy_loss(q, c), torch.tensor(1.2432), atol=1e-4)

test_prenormalized_cross_entropy_loss()

This assumed that `q` was a probability distribution, but usually neural networks output logits $l$, which are unnormalized probabilities.
One way we could normalize our logits $l$ into probabilities $q$ is to divide $l$ by the sum of logits $q_i = \frac{l_i}{\sum_j l_j}$, but that doesn't work if $l_i$ is negative.
The softmax, which exponentiates the $logit$, $q_i = \frac{e^{l_i}}{\sum_j e^{l_j}}$, before dividing by the sum of exponentiated logits removes the issues of negativity (there are other good reasons for using softmax, such as numerical stability).

However, applying `q = softmax(l)` to `prenormalized_cross_entropy_loss(q, c)` can still be numerically unstable.
Luckily, we can simplify our function:
$$\begin{align}
CE(l, c) &= - \log \frac{e^{l_c}}{\sum_j e^{l_j}} \\
&= - (\log e^{l_c} - \log \sum_j e^{l_j}) \\
&= - (l_c - \log \sum_j e^{l_j}) \\
&= - l_c + \log \sum_j e^{l_j}
\end{align}$$

While you could implement $\log \sum_j e^{l_j}$, you should use `torch.logsumexp()` which will exponentiate, sum, and then log your logits, but in a more numerically stable way.
Implement `cross_entropy_loss()` below.
You can validate it works by comparing it with the output of `F.cross_entropy()`.

*Note: Do **not** use a for loop*

In [ ]:
def cross_entropy_loss(l, c):
    pass

It is hard to tell how well a model is performing just from its cross entropy loss, so create a `get_accuracy()` function to measure accuracy.
`get_accuracy()` takes in a `y_hat` and `y`, where `y_hat` contains the predicted logits (unnormalized probabilities) for some images `x` and `y` are the labels.
You can get the predicted label from `y_hat`, by using the `torch.argmax()` function.

*Note: Do **not** use a for loop.*

In [ ]:
def get_accuracy(y_hat, y):
    pass

## Validation and Training

To see how well training is going, implement a `validation()` function to compute the average loss and accuracy over all instances in the `val_loader`. 
This function will look very similar to a basic training loop, but without any optimization, e.g. no `loss.backward()` or `optimizer.step()`. 
To speed up the process use `torch.no_grad()` to keep PyTorch from building the computation graph.
You can use `torch.no_grad()` either as a decorator:
```python
@torch.no_grad() 
def fn():
    ...
```
or as a context manager:
```python
def fn(): 
    with torch.no_grad():
        ...
```

*Remember you can use `.item()` on a tensor with one element to convert it into a float/int.*

In [ ]:
def validation(net, val_loader):
    # TODO: Return the network's average loss and accuracy on the val loader
    pass

Now implement the `train()` function.
This will look similar to the `train()` you implemented in lab 2, but now you will also store training accuracies, and at every `log_val_interval` you will call `validation` and store the validation loss and accuracy.

In [ ]:
def train(net, train_loader, val_loader, optimizer, n_optimization_steps, log_val_interval):
    # TODO: Implement training loop and return the training and validation losses and accuracies.
    pass

Now train an `MLP` on `MNIST`. To speed up training use the `torch.optim.Adam` optimizer instead of `torch.optim.SGD`.

*Note: Don't forget to put your network on `device`.*

In [ ]:
# TODO: Set torch seed for reproducibility

# TODO: Initialize your MLP, called net
net = None

# TODO: Create an Adam optimizer (lr=.001 works well)

# TODO: Train your MLP for 2000 steps and set log_val_interval=50


Now plot your training and validation loss on the same plot and plot your training and validation accuracies on the same plot.
Properly set your x- and y-axis labels and create a legend to make your plot legible.  

*Note: that you can specify the x-values for each point by calling `plt.plot(x, y)` instead of `plt.plot(y)`. Since you store validation every 50 steps, you'll need to use `torch.arange` to get the proper x-values to align the validation results to the training results.*

___

# Convolution Networks

Now we will create a convolution network.
When we were dealing with an `MLP` we used fully-connected `nn.Linear` layers.
In a convolution network we use `nn.Conv2d` layers.
`nn.Linear` maps tensors of shape `(B, F_in) -> (B, F_out)`.
`nn.Conv2d` maps tensors of shape `(B, C_in, H_in, W_in) -> (B, C_out, H_out, W_out)`, where `C_in` represents our input channels and `C_out` represents our output channels.
You decide what `C_out` should be when you initialize.
The mapping that

If we had a batch of `img` tensors with shape `(B, 3, 8, 8)` and we wanted it to become `(B, 6, 4, 4)` we could create a convolution layer:
```python
conv_layer = nn.Conv2d(in_channels=3,  # Our 'C_in' which is 3
                       out_channels=6, # Our desired 'C_out'
                       kernel_size=2,  # One way to make an 8x8 image become a 2x2 image is to have the kernel be 2x2,
                       padding=0,      #    with zero padding,
                       stride=2,       #    and a stride of 2.
                      )
```
Validate this is true below by creating `conv_prac()` function, which creates a random tensor with shape `(B, 3, 8, 8)`, passes it through `conv_layer`, and prints out the resulting shape.

*Note: `kernel_size`, `padding`, and `stride` can all be tuples in case you want different (height, width) parameters, e.g. `kernel=(kernel_height, kernel_width)`.*

In [ ]:
def conv_prac():
    pass

conv_prac()

___

### Quiz
Test your knowledge of how convolution layers affect the shape of outputs by answering the following quiz questions.


*Using a Kernel size of 3×3 what should the settings of your 2d convolution be that results in the following mappings (first answer given to you)*

* (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : **(out_channels=10, kernel_size=(3, 3), padding=(0, 0))**
* (c=3, h=10, w=10) ⇒ (c=22, h=10, w=10) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=65, h=12, w=12) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=7, h=20, w=20) : **Your answer in bold here**

*Using a Kernel size of 5×5:*

* (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : (out_channels=10, kernel_size=(5, 5), padding=(1, 1))
* (c=3, h=10, w=10) ⇒ (c=100, h=10, w=10) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=23, h=12, w=12) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=5, h=24, w=24) : **Your answer in bold here**

*Using Kernel size of 5×3:*

* (c=3, h=10, w=10) ⇒ (c=10, h=8, w=8) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=100, h=10, w=10) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=23, h=12, w=12) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=5, h=24, w=24) : **Your answer in bold here**

*Determine the kernel that requires the smallest padding size to make the following mappings possible:*

* (c=3, h=10, w=10) ⇒ (c=10, h=9, w=7) : **Your answer in bold here**
* (c=3, h=10, w=10) ⇒ (c=22, h=10, w=10) : **Your answer in bold here**




## ConvNet

Now create a convolution network `ConvNet` that inherits `nn.Module`.
The network should have 3 convolution layers (each layer should have 16 output channels):
1. Conv Layer 1 should have a 6x6 kernel, no padding, and a stride of 2.
2. Conv Layer 2 should have a 4x4 kernel, no padding, and a stride of 2.
3. Conv Layer 3 should have a 3x3 kernel, no padding, and a stride of 1.

The output of these layers should be a 3x3 image with 16 channels.
You should flatten the image (you can use `.view()`, `torch.flatten()`, or `nn.Flatten()`) and then pass it through 2 linear layers:
1. Linear Layer 1 should take the flattened image and map it to a vector with 16 features.
2. Linear Layer 2 should map its vector to logits.

Do not forget to add nonlinearities between the layers (do not add them to the last layer).

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, ):
        pass

    def forward(self, x):
        pass

Train you convolution network below.

In [ ]:
# TODO: Set seed for reproducibility

# TODO: Initalize your ConvNet, called 'conv_net'
conv_net = None

# TODO: Create an Adam optimizer (lr=.001 works well)

# TODO: Train your ConvNet for 2000 steps and set log_val_interval=50


Now plot your training and validation loss on the same plot and plot your training and validation accuracies on the same plot.

Create a function that outputs the number of parameters in a network.
Remember you can call `.parameters()` to recursively retrieve the `Parameter`s in a `Module`.
You could then use `.shape` to figure out the number of parameters in a `Parameter` or you could flatten the parameter and get its length.

In [ ]:
def get_n_net_params(net):
    pass

Print the number of parameters and the accuracy of your trained `MLP` and `ConvNet` networks and then write down below:
- Which one is more accurate?
- Which one is smaller?

*Note: You can use `validation()` to get the accuracy of your network.*